In [7]:
# import os

# # Define the cache directory
# cache_dir = './.pip_cache'

# # Check if the cache directory exists, if not, create it
# if not os.path.exists(cache_dir):
#     os.makedirs(cache_dir)

# def install_package(package_name):
#     # Check if the package is already installed
#     try:
#         __import__(package_name)
#         print(f"{package_name} is already installed")
#     except ImportError:
#         # Install the package
#         !pip install --no-cache-dir {package_name}
#         print(f"{package_name} has been installed")

# # List of packages to install
# packages = [
#     'langchain==0.0.350',
#     'accelerate==0.25.0',
#     'transformers==4.36.2',
#     'bitsandbytes==0.41.2',
#     'sentence-transformers==2.2.2',
#     'InstructorEmbedding==1.0.1',
#     'pymupdf==1.23.8',
#     'faiss-gpu==1.7.2',
#     'faiss-cpu==1.7.4'
# ]

# # Install each package
# for package in packages:
#     install_package(package)

In [1]:
import os
import torch

device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

**Prompt**

In [2]:
from langchain import PromptTemplate

prompt_template = """
    Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.
    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.
    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.
    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template='Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.\n    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.\n    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.\n    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.\n    {context}\n    Question: {question}\n    Answer:')

In [3]:
PROMPT.format(
    context = """The Asian Institute of Technology (AIT), founded in 1959, is an international organization for higher education situated 40km north of Bangkok, Thailand.
    It specializes in engineering, advanced technologies, sustainable development, and management and planning.""",
    question = "What is AIT"
)

'Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.\n    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.\n    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.\n    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.\n    The Asian Institute of Technology (AIT), founded in 1959, is an international organization for higher education situated 40km north of Bangkok, Thailand.\n    It specializes in engineering, advanced technologies, sustainable development, and management and planning.\n    Question: What is AIT\n    Answer:'

In [4]:
# !pip install langchain==0.1.6
# !pip install langchain-community==0.0.19

**Retrieval**

2.1 Document Loader

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

In [6]:
nlp_docs = './SET-ICT-DSAI.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [7]:
len(documents)

2

2.2 Docment Transformer

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 70,
    chunk_overlap = 10
)

doc = text_splitter.split_documents(documents)

In [9]:
doc[0]

Document(page_content='Data Science (DS) is concerned with extraction', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''})

2.3 Text Embedding Models

In [10]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

C:\Users\user\anaconda3\envs\A7\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
C:\Users\user\anaconda3\envs\A7\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


C:\Users\user\anaconda3\envs\A7\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\user\anaconda3\envs\A7\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


2.4 Vector Stores

In [11]:
#locate vectorstore
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [12]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

2.5 Retrievers

In [13]:
from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [14]:
#ready to use
retriever = vectordb.as_retriever()

In [15]:
retriever.get_relevant_documents("What is DSAI")

[Document(page_content='•\n•\n•\nhttps://www.dsai-project.eu/', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
 Document(page_content='To study DS&AI, students should come from one of the following', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
 Document(page_content='SCHOOL OF ENGINEERING AND TECHNOLOGY\nDSAI Webpage\nSET Webpage', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': '

In [16]:
retriever.get_relevant_documents("What is NLU")

[Document(page_content='at both l\nat both levels.\nABOUT THE PROGRAM\nPROGRAM STRUCTURE', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
 Document(page_content='established fields of machine learning, data', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
 Document(page_content='of useful knowledge from data sets.  It is closely', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 

**Memory**

In [17]:
# !pip install langchain langsmith==0.1.2

In [18]:
# !pip install --upgrade langchain

In [19]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

**Chain**

In [21]:
# import os

# # Clone the repository
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

# # Specify the repository name and directory
# repository_name = 'fastchat-t5-3b-v1.0'
# repository_path = f'./{repository_name}'

# # Move the cloned repository to the desired directory
# !mv {repository_name} {repository_path}

# # Change the current directory to the repository directory
# os.chdir(repository_path)

In [22]:
# !pip install protobuf

In [23]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = 'fastchat-t5-3b-v1.0'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit = False,
#     bnb_4bit_quant_type = "nf4",
#     bnb_4bit_compute_dtype = torch.float16,
#     bnb_4bit_use_double_quant = False
# )

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id
    # quantization_config = bitsandbyte_config, #caution Nvidia
    # device_map = 'cpu',
    # load_in_8bit = False
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
# from langchain_core.runnables.utils import create_model

from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [25]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [26]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [27]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

C:\Users\user\anaconda3\envs\A7\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  are  the  main  differences  between  Machine  Learning  and  Deep  Learning  AI?\n'}

In [28]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.\n    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.\n    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.\n    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x0000019C7BA5D8E0>)), document_variable_name='context')

In [29]:
query = "What is AIT"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.
    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.
    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.
    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.
    AIT Website

Asian Institute of
Asian Institute of Technology (AIT)

At AIT, we will prepare you to understand and

science and AI? AIT is ready to help
    Question: What is AIT
    Answer:

> Finished chain.

> Finished chain.


{'input_documents': [Document(page_content='AIT Website', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
  Document(page_content='Asian Institute of\nAsian Institute of Technology (AIT)', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
  Document(page_content='At AIT, we will prepare you to understand and', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'autho

In [30]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.\n    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.\n    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.\n    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 

In [31]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Founded in 1959 as SEATO Graduate School of Engineering, it receives funding from organizations and governments around the world.
    In 1967, The Constituent Assembly of Thailand approved legislation for the Charter of the newly named Asian Institute of Technology in October.
    The Asian Institute of Technology Enabling Act was published in the Royal Thai Government Gazette in November the same year.
    AIT became independent of SEATO as an institution of higher learning empowered to grant degrees.
    Are you ready for the age of data

backgrounds.
Computer Science / Computer Engineering / ICT.

of 
human 
endeavor, transforming the way we live

and work. In the longer term, Artificial
    Question: Who are you by the way?
    Answer:

> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad> Answer:  I  am  a  student  at  AIT.\n',
 'source_documents': [Document(page_content='Are you ready for the age of data', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
  Document(page_content='backgrounds.\nComputer Science / Computer Engineering / ICT.', metadata={'source': './SET-ICT-DSAI.pdf', 'file_path': './SET-ICT-DSAI.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.1.1', 'producer': 'www.ilovepdf.com', 'creationDate': '', 'modDate': 'D:20200309103917Z', 'trapped': ''}),
  Document(page_content='of \nhuman \nendeavor, transforming the way we

In [32]:
answer['answer']

'<pad> Answer:  I  am  a  student  at  AIT.\n'